In [ ]:
!pip install fuzzywuzzy

In [ ]:
import pandas as pd
import re
import math
from unidecode import unidecode
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [ ]:
# reading dataframe

df = pd.read_excel("file.xlsx")

In [ ]:
# taking column which interests us
df['pierwsza_kolumna']

In [ ]:
#creating lists from this column

column = list(df['pierwsza_kolumna'])


In [ ]:
#printing type of the values in the list

for i in column:
    print(type(i))

In [ ]:
# printing elements from list


for i in column:
    print(i)

In [ ]:
# when there is Nan value we add "Brak_wartosci" and where is different value we convert it to a string


column_str = []

for i in column:
    if type(i) is float:
        column_str.append("BRAK_WARTOSCI")
    else:
        column_str.append(str(i))

In [ ]:
# we change spaces for commas

new_list = []


for item in column_str:

    updated_string = re.sub(r'\s+', ',', item)
    new_list.append(updated_string)
    
new_list

In [ ]:
# we delete additional commas

poj_przecinki = []


for item in new_list:

    updated_item = re.sub(r',+', ',', item)
    poj_przecinki.append(updated_item)
    
poj_przecinki

In [ ]:

# creating dictionary to add value and how many it occured


value_counts = {}


for item in poj_przecinki:
    
    # if the value is not nan we do it
    if isinstance(item, str) and item != 'nan':
        
        # we strip, make the value lower and unicode those value in ASCII mode to be able to compare in the best way values to each other

        values = [unidecode(value.strip().lower()) for value in item.split(',')]
        
        
        # we calculate how many times the value has occured
        for value in values:
            value_counts[value] = value_counts.get(value, 0) + 1


for value, count in value_counts.items():
    print(f"{value}: {count}")


In [ ]:
wszystkie_wartosci = []


# creating list of every value which occured in answers
for value, count in value_counts.items():
    wszystkie_wartosci.append(value)
wszystkie_wartosci

In [ ]:
# create list which contains the list for the answers of each respondent


nazwy_badanych = []



for item in poj_przecinki:
    if isinstance(item, str):
        # we change the name of answers for each respondent to be compatible with all the values which respondents made
        values = [unidecode(value.strip().lower()) for value in item.split(',')]
        
        nazwy_badanych.append(values)

# this is a list from the all answers for each respondent, the names are cleaned to be compatible with all the values
len(nazwy_badanych)

In [ ]:
nazwy_badanych

In [ ]:

# we group similar names and add it to the list similar_names

threshold = 70

similar_names = []


# we loop from all the values which occured in answers
for name in wszystkie_wartosci:
    
    # we compare the specific name and assign it to a correct group of similar names
    
    best_match = process.extractOne(name, similar_names)
    
    if best_match is not None and best_match[1] > threshold:
        
        similar_names[similar_names.index(best_match[0])].append(name)
        
    else:
        similar_names.append([name])
        
for group in similar_names:
    print(group)

In [ ]:
similar_names

In [ ]:
dict_of_similar_names = {
    "you need to create dictionary in python from the similar group words manually based on similar_names list"
}

In [ ]:


# creating two lists of keys and values from dictionary of specific names
keys = []
values = []

for key, value in dict_of_similar_names.items():
    keys.append(key)
    values.append(value)
    

In [ ]:
values

In [ ]:
# we create an empty dataframe where the columns are the keys (the name of each group) to which we will assign the answers from each respondent


df_to_modify = pd.DataFrame(columns=keys, index=range(len(nazwy_badanych)))
df_to_modify

In [ ]:
def get_key_from_value(dictionary, value):
    # function which look for a value and return the proper key for those value
    
    # this function will be necessery to find a value which will be the answer from respondent and find a key
    # in order to add the 1 to the proper column in the dataframe (proper name of the key)
    # to know which category was used by the respondent
    
    
    for key, values in dictionary.items():
        if value in values:
            return key
    return None

In [ ]:



# creating a loop to add the number 1 when the specific category occured in the respondent's answer
for index, row in df_to_modify.iterrows():
    if nazwy_badanych[index] != 'nan':
        
        # we loop for the answer of each respondent
        for nazwa in nazwy_badanych[index]:
            
            # if the answer is in the all the values we then...
            if nazwa in wszystkie_wartosci:
                # we then take the key from our dictionary of similar names to be able to find this column in dataframe
                # because the names of the columns are the keys from that dictionary
                nazwa_kolumny = get_key_from_value(dict_of_similar_names,nazwa)
                
                
                # we then add 1 to that column in dataframe because that category has occured in the respondent's answer
                # so if the category has occured there is an addition 1 to a specific column which means the category
                
                # so it finds the concrete value from the respondent's answer and add 1 to a proper category
                
                print(nazwa_kolumny)
                row[nazwa_kolumny] = 1
            

In [ ]:
# saving the dataframe which contains the category of the respondent's answers. When the category occured in the answer in the dataframe
# the number 1 signify it

df_to_modify.to_excel("answers_category.xlsx")